In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# General Overview of Datasets
## Familiarize with the data

In [ ]:
measurement_df = pd.read_csv("Resources/hawaii_measurements.csv")
station_df = pd.read_csv("Resources/hawaii_stations.csv")

In [ ]:
measurement_df.info()

In [ ]:
station_df.info()

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, distinct

In [ ]:
#Creating a database engine
#dialact object = hawaii.sqlite 
#no pool created here, but is possible
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()

In [ ]:
# reflect an existing database into a new model - brings the ORM
Base = automap_base()

# reflect the tables
Base.prepare(engine,reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

# station = pd.read_sql("SELECT * FROM station", conn)
# measurement = pd.read_sql("SELECT * FROM measurement", conn)

In [ ]:
# Create our session (link) from Python to the ORM which is linked to the base linked to connection and engine 
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
#Get the last date in the dataset
last_date = session.query(measurement.date).order_by(measurement.date.desc()).first()[0]

#Convert query result (string) to datetime
last_date = dt.datetime.strptime(str(last_date), "%Y-%M-%d")

# Calculate the date 1 year ago from the last data point in the database
one_year = last_date - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
query = session.query(measurement.date, measurement.station, measurement.prcp, measurement.tobs).filter(measurement.date > one_year)

# def to_dict(self):
#     return {column.name: getattr(self, column.name)
#         if not isinstance(getattr(self, column.name), datetime.datetime)
#         else getattr(self, column.name).isoformat()
#         for column in self.__table__.columns}

# [x._asdict() for x in query]
one_df = pd.DataFrame(query)

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
one_df["prcp"].describe()

In [ ]:
# Design a query to show how many stations are available in this dataset? 
#Pandas version
tot_stat = one_df["station"].nunique()
print(f"There are {tot_stat} stations in the dataset.")

In [265]:
# Design a query to show how many stations are available in this dataset? 
#SQLAlchmey version
x = session.query(distinct(measurement.station))\
            .filter(measurement.date > one_year)\
            .count()
print(f"There are {x} weather stations in the Hawaii dataset.")


There are 7 weather stations in the Hawaii dataset.


In [266]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
stat_rain = one_df.groupby(["station"])\
                    .count()["date"]\
                    .sort_values(ascending=False)
stat_rain = pd.DataFrame(stat_rain).reset_index()
top_station = stat_rain.iloc[0]["station"]
top_counts = stat_rain.iloc[0]["date"]

print(f"The most active weather station is {top_station} with {top_counts} recordings")

The most active weather station is USC00519397 with 572 recordings


In [284]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
low_top_temp = session.query(measurement.tobs)\
            .filter(measurement.date > one_year)\
            .filter(measurement.station == top_station)\
            .order_by(measurement.tobs).first()

high_top_temp = session.query(measurement.tobs)\
            .filter(measurement.date > one_year)\
            .filter(measurement.station == top_station)\
            .order_by(measurement.tobs.desc()).first()

avg_top_temp = session.query(func.avg(measurement.tobs))\
            .filter(measurement.date > one_year)\
            .filter(measurement.station == top_station).first()

#Convert results to dataframe
low_top_temp = pd.DataFrame(low_top_temp).reset_index()
low_top_temp = low_top_temp.iloc[0][0]

#Convert results to dataframe
high_top_temp = pd.DataFrame(high_top_temp).reset_index()
high_top_temp = high_top_temp.iloc[0][0]

#Convert results to dataframe
avg_top_temp = pd.DataFrame(avg_top_temp).reset_index()
avg_top_temp = avg_top_temp.iloc[0][0]



In [296]:

print(f"Temperature recordings for weather station {top_station}")
print("------------------------------------------------------")
print(f"Minumum Temperature: {low_top_temp}")
print(f"Maximum Temperture: {high_top_temp}")
print(f"Average Temperature: {round(avg_top_temp,1)}")

Temperature recordings for weather station USC00519397
------------------------------------------------------
Minumum Temperature: 60.0
Maximum Temperture: 87.0
Average Temperature: 75.0


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
